In [3]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'pyspark-shell'

In [4]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [6]:
df_csv = spark.read.csv("autos.csv", header=True, inferSchema=True)

In [9]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: string (nullable = true)
 |-- name: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- dateCreated: string (nullable = true)
 |-- nrOfPictures: integer (nullable = true)
 |-- postalCode: string (nullable = true)
 |-- lastSeen: timestamp (nullable = true)



In [10]:
df_csv.count()

371824

In [11]:
df_csv_duplicates=df_csv.dropDuplicates()
df_csv_duplicates.count()

371824

In [15]:
df_csv.columns


['index',
 'dateCrawled',
 'name',
 'seller',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'dateCreated',
 'nrOfPictures',
 'postalCode',
 'lastSeen']

In [18]:
df_csv.select('price').describe().show()

+-------+------------------+
|summary|             price|
+-------+------------------+
|  count|            371823|
|   mean|17295.124087885943|
| stddev|  3587958.57312194|
|    min|                 0|
|    max|              test|
+-------+------------------+



In [22]:
df_csv.filter("vehicleType is NULL").show()

+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|index|        dateCrawled|                name|seller|offerType|price| abtest|vehicleType|yearOfRegistration|gearbox|powerPS|   model|kilometer|monthOfRegistration|fuelType|     brand|notRepairedDamage|        dateCreated|nrOfPictures|postalCode|           lastSeen|
+-----+-------------------+--------------------+------+---------+-----+-------+-----------+------------------+-------+-------+--------+---------+-------------------+--------+----------+-----------------+-------------------+------------+----------+-------------------+
|    0|2016-03-24 11:52:17|          Golf_3_1.6|privat|  Angebot|  480|   test|       null|              1993|manuell|      0|    golf|   150000|                  0|  benzin|volkswagen|           

In [24]:
df_csv=df_csv.drop('nrOfPictures','seller','lastSeen','dateCreated')

In [25]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'offerType',
 'price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [29]:
df_csv.select('brand').distinct().show()

+-------------+
|        brand|
+-------------+
|       jaguar|
|         nein|
|     daihatsu|
|   mitsubishi|
|         null|
|         lada|
|       toyota|
|         seat|
|         saab|
|   land_rover|
|      peugeot|
|     chrysler|
|      citroen|
|         audi|
|mercedes_benz|
|          bmw|
|         jeep|
|       lancia|
|        skoda|
|        rover|
+-------------+
only showing top 20 rows



In [30]:
from pyspark.sql.functions import to_timestamp

In [33]:
df_csv = df_csv.withColumn('dateCrawled', to_timestamp(df_csv["dateCrawled"], "yyyy-MM-dd HH:mm:ss"))


In [35]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [37]:
df_csv.select('dateCrawled').show(5)

+-------------------+
|        dateCrawled|
+-------------------+
|2016-03-24 11:52:17|
|2016-03-24 10:58:45|
|2016-03-14 12:52:21|
|2016-03-17 16:54:04|
|2016-03-31 17:25:20|
+-------------------+
only showing top 5 rows



In [38]:
from pyspark.sql.functions import to_timestamp, year
df_csv = df_csv.withColumn("dateCrawled", year(df_csv["dateCrawled"]))

In [39]:
df_csv.select('dateCrawled').show(5)

+-----------+
|dateCrawled|
+-----------+
|       2016|
|       2016|
|       2016|
|       2016|
|       2016|
+-----------+
only showing top 5 rows



In [40]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: string (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [42]:
df_csv.select('yearOfRegistration').show(10)

+------------------+
|yearOfRegistration|
+------------------+
|              1993|
|              2011|
|              2004|
|              2001|
|              2008|
|              1995|
|              2004|
|              1980|
|              2014|
|              1998|
+------------------+
only showing top 10 rows



In [46]:
from pyspark.sql.functions import col, when
df_csv = df_csv.withColumn('yearOfRegistration', col('yearOfRegistration').cast("int"))

In [47]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- price: string (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [48]:
df_csv = df_csv.withColumn('Price', col('Price').cast("float"))

In [49]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: string (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [50]:
df_csv = df_csv.withColumn('monthOfRegistration', col('monthOfRegistration').cast("int"))

In [57]:
df_csv.printSchema()

root
 |-- index: string (nullable = true)
 |-- dateCrawled: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- offerType: string (nullable = true)
 |-- Price: float (nullable = true)
 |-- abtest: string (nullable = true)
 |-- vehicleType: string (nullable = true)
 |-- yearOfRegistration: integer (nullable = true)
 |-- gearbox: string (nullable = true)
 |-- powerPS: string (nullable = true)
 |-- model: string (nullable = true)
 |-- kilometer: integer (nullable = true)
 |-- monthOfRegistration: integer (nullable = true)
 |-- fuelType: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- notRepairedDamage: string (nullable = true)
 |-- postalCode: string (nullable = true)



In [58]:
correlation_value2_value3 = df_csv.stat.corr("monthOfRegistration", "Price")
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: -0.0028176063434703284


In [59]:
correlation_value2_value3 = df_csv.stat.corr("dateCrawled", "Price")
print(f"Correlation between value2 and value3: {correlation_value2_value3}")

Correlation between value2 and value3: 0.00013600453977892399


In [63]:
df_csv.select("monthOfRegistration").distinct().show()

+-------------------+
|monthOfRegistration|
+-------------------+
|                 12|
|               null|
|                  1|
|                  6|
|                  3|
|                  5|
|                  9|
|                  4|
|                  8|
|                  7|
|                 10|
|                 11|
|                  2|
|                  0|
+-------------------+



In [64]:
df_csv=df_csv.drop('offerType')

In [65]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [68]:
zero_count = df_csv.filter(col("Price") == 0).count()
print(zero_count)

10779


In [69]:
df_csv.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [103]:
from pyspark.sql.functions import col, sum
df=df_csv
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|      296|
|               name|        0|
|              Price|      297|
|             abtest|       30|
|        vehicleType|    37871|
| yearOfRegistration|      297|
|            gearbox|    20211|
|            powerPS|       15|
|              model|    20485|
|          kilometer|        1|
|monthOfRegistration|      297|
|           fuelType|    33388|
|              brand|       64|
|  notRepairedDamage|    72061|
|         postalCode|        1|
+-------------------+---------+



In [76]:
df.select('Price').describe().show()

+-------+------------------+
|summary|             Price|
+-------+------------------+
|  count|            371527|
|   mean|17295.124128259857|
| stddev|3587958.5758335786|
|    min|               0.0|
|    max|      2.14748365E9|
+-------+------------------+



In [94]:
# Calculate the 5th and 95th percentiles
quantiles = df.approxQuantile("price", [0.05, 0.95], 0.01)

# Extract the 5th and 95th percentiles
quantile_05 = quantiles[0]
quantile_95 = quantiles[1]

print(f"5th percentile: {quantile_05}")
print(f"95th percentile: {quantile_95}")

5th percentile: 250.0
95th percentile: 19500.0


In [104]:
df_copy=df_csv

In [105]:
df_copy = df_copy.filter(col("price") != 0)


In [106]:

df=df_copy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|    34127|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [107]:
most_common_row = df_copy.groupBy("vehicleType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["vehicleType"]
df_VehicleCopy = df_copy.fillna({"vehicleType": most_common})

In [108]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|    17729|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [110]:
value_counts = df_VehicleCopy.groupBy("vehicleType").count()

# Step 2: Sort by count in descending order
sorted_value_counts = value_counts.orderBy(col("count").desc())
sorted_value_counts.show()

+-----------+------+
|vehicleType| count|
+-----------+------+
|  limousine|127969|
| kleinwagen| 78209|
|      kombi| 66098|
|        bus| 29769|
|     cabrio| 22560|
|      coupe| 18487|
|        suv| 14501|
|     andere|  3155|
+-----------+------+



In [111]:
most_common_row =df_VehicleCopy.groupBy("gearbox").count().orderBy(col("count").desc()).first()
most_common = most_common_row["gearbox"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"gearbox": most_common})

In [112]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|    29948|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [113]:
most_common_row =df_VehicleCopy.groupBy("fuelType").count().orderBy(col("count").desc()).first()
most_common = most_common_row["fuelType"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"fuelType": most_common})

In [115]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|    66769|
|         postalCode|        0|
+-------------------+---------+



In [116]:
most_common_row =df_VehicleCopy.groupBy("notRepairedDamage").count().orderBy(col("count").desc()).first()
most_common = most_common_row["notRepairedDamage"]

# Step 2: Replace missing values in 'gearbox' column with the most common value
df_VehicleCopy = df_VehicleCopy.fillna({"notRepairedDamage": most_common})

In [117]:
df=df_VehicleCopy
# Get the list of columns
columns = df.columns

# Create a dictionary to store the null count for each column
null_counts = {col_name: df.filter(col(col_name).isNull()).count() for col_name in columns}

# Convert the dictionary to a DataFrame for better display
null_counts_df = spark.createDataFrame([(k, v) for k, v in null_counts.items()], ["Column", "NullCount"])

# Show the null counts
null_counts_df.show()

+-------------------+---------+
|             Column|NullCount|
+-------------------+---------+
|              index|        0|
|        dateCrawled|        0|
|               name|        0|
|              Price|        0|
|             abtest|        0|
|        vehicleType|        0|
| yearOfRegistration|        0|
|            gearbox|        0|
|            powerPS|        0|
|              model|    18300|
|          kilometer|        0|
|monthOfRegistration|        0|
|           fuelType|        0|
|              brand|        0|
|  notRepairedDamage|        0|
|         postalCode|        0|
+-------------------+---------+



In [121]:
df_VehicleCopy.columns

['index',
 'dateCrawled',
 'name',
 'Price',
 'abtest',
 'vehicleType',
 'yearOfRegistration',
 'gearbox',
 'powerPS',
 'model',
 'kilometer',
 'monthOfRegistration',
 'fuelType',
 'brand',
 'notRepairedDamage',
 'postalCode']

In [123]:
#df = spark.createDataFrame(df_VehicleCopy, df_VehicleCopy.columns)
pandas_df = df_VehicleCopy.toPandas()

In [127]:
pip install openpyxl


    100% |████████████████████████████████| 256kB 391kB/s ta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [128]:
import openpyxl


In [129]:
file_path = "/home/talentum/shared/Project_Data_Cleaned/output.xlsx"

# Write Pandas DataFrame to an Excel file
pandas_df.to_excel(file_path, index=False,engine='openpyxl')
